<a href="https://colab.research.google.com/github/mralamdari/CV-Yolo/blob/main/YOLO_v3_ObjectDetection_TensorFlow_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

!wget -P model_data https://pjreddie.com/media/files/yolov3.weights

--2022-08-15 14:12:02--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘model_data/yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  39.7MB/s    in 6.4s    

2022-08-15 14:12:09 (37.0 MB/s) - ‘model_data/yolov3.weights’ saved [248007048/248007048]



In [2]:
with open('model_data/yolov3.weights', 'rb') as wf:
    major, minor, revision, seen, _ = np.fromfile(wf, dtype=np.int32, count=5)

    weights_1 = np.fromfile(wf, dtype=np.float32, count=2)
    weights_2 = np.fromfile(wf, dtype=np.float32, count=20)
    weights_3 = np.fromfile(wf, dtype=np.float32)




In [3]:
major, minor, revision, seen, _ 

(0, 2, 0, 32013312, 0)

In [4]:
weights_1

array([-4.316885 , -0.7578076], dtype=float32)

In [5]:
weights_2

array([-2.1098018 ,  1.7402638 ,  1.4071269 , -3.0952053 , -0.38860837,
        0.75603795,  1.982805  ,  1.2893223 ,  0.652888  ,  2.6263633 ,
        2.3013082 , -2.04827   , -3.7340226 , -2.0467598 ,  3.845535  ,
       -1.0419698 , -0.30135924, -0.35420752, -3.5354283 , -2.628548  ],
      dtype=float32)

In [6]:
weights_3

array([ 0.74821305,  0.3917957 ,  2.3627155 , ..., -0.03373355,
       -0.04969028, -0.11880384], dtype=float32)

In [7]:
weights_3.shape

(62001735,)

In [8]:
import tensorflow as tf

# tf.config.experimental.list_physical_devices('GPU')
tf.config.list_physical_devices('GPU')

[]

In [9]:
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    try: 
      tf.config.set_memory_growth(physical_devices[0], True)
    except RuntimeError:
      pass

In [10]:
X = tf.random.normal([512, 32]);
Y = tf.split(X,128,0)
Y_shape = tf.shape(Y[0])
Y_shape2 = tf.shape(Y[1])


print(len(X))
print(len(Y))
print(Y_shape)
print(Y_shape2)


512
128
tf.Tensor([ 4 32], shape=(2,), dtype=int32)
tf.Tensor([ 4 32], shape=(2,), dtype=int32)


In [11]:
X.shape

TensorShape([512, 32])

In [12]:
a=tf.random_normal_initializer(stddev=0.01)

In [13]:
s=tf.constant_initializer(0.)

In [14]:
a

In [15]:
s

In [17]:
a.shape

In [19]:
s.shape

In [20]:
output_size = 20
y = tf.range(output_size, dtype=tf.int32)
y = tf.expand_dims(y, -1)
y = tf.tile(y, [1, output_size])
x = tf.range(output_size,dtype=tf.int32)
x = tf.expand_dims(x, 0)
x = tf.tile(x, [output_size, 1])

xy_grid = tf.meshgrid(tf.range(output_size), tf.range(output_size))
xy_grid = tf.expand_dims(tf.stack(xy_grid, axis=-1), axis=2)  # [gx, gy, 1, 2]
xy_grid = tf.tile(tf.expand_dims(xy_grid, axis=0), [32, 1, 1, 3, 1])
xy_grid = tf.cast(xy_grid, tf.float32)

In [30]:
y = tf.range(output_size, dtype=tf.int32)
y

<tf.Tensor: shape=(20,), dtype=int32, numpy=
array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19], dtype=int32)>

In [31]:
y = tf.expand_dims(y, -1)
y

<tf.Tensor: shape=(20, 1), dtype=int32, numpy=
array([[ 0],
       [ 1],
       [ 2],
       [ 3],
       [ 4],
       [ 5],
       [ 6],
       [ 7],
       [ 8],
       [ 9],
       [10],
       [11],
       [12],
       [13],
       [14],
       [15],
       [16],
       [17],
       [18],
       [19]], dtype=int32)>

In [32]:
y = tf.tile(y, [1, output_size])
y

<tf.Tensor: shape=(20, 20), dtype=int32, numpy=
array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0],
       [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1],
       [ 2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  2],
       [ 3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
         3,  3,  3,  3],
       [ 4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
         4,  4,  4,  4],
       [ 5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
         5,  5,  5,  5],
       [ 6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
         6,  6,  6,  6],
       [ 7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
         7,  7,  7,  7],
       [ 8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
         8,  8,  8,  8],
       [ 9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
       

In [33]:
x = tf.range(output_size,dtype=tf.int32)
x = tf.expand_dims(x, 0)
x

<tf.Tensor: shape=(1, 20), dtype=int32, numpy=
array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16, 17, 18, 19]], dtype=int32)>

In [34]:
x = tf.tile(x, [output_size, 1])
x

<tf.Tensor: shape=(20, 20), dtype=int32, numpy=
array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16, 17, 18, 19],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16, 17, 18, 19],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16, 17, 18, 19],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16, 17, 18, 19],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16, 17, 18, 19],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16, 17, 18, 19],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16, 17, 18, 19],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16, 17, 18, 19],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16, 17, 18, 19],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
       

In [26]:
xy_grid = tf.meshgrid(tf.range(output_size), tf.range(output_size))
xy_grid

[<tf.Tensor: shape=(20, 20), dtype=int32, numpy=
 array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
         16, 17, 18, 19],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
         16, 17, 18, 19],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
         16, 17, 18, 19],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
         16, 17, 18, 19],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
         16, 17, 18, 19],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
         16, 17, 18, 19],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
         16, 17, 18, 19],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
         16, 17, 18, 19],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
         16, 17, 18, 19],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12,

In [27]:
xy_grid = tf.expand_dims(tf.stack(xy_grid, axis=-1), axis=2)  # [gx, gy, 1, 2]
xy_grid

<tf.Tensor: shape=(20, 20, 1, 2), dtype=int32, numpy=
array([[[[ 0,  0]],

        [[ 1,  0]],

        [[ 2,  0]],

        [[ 3,  0]],

        [[ 4,  0]],

        [[ 5,  0]],

        [[ 6,  0]],

        [[ 7,  0]],

        [[ 8,  0]],

        [[ 9,  0]],

        [[10,  0]],

        [[11,  0]],

        [[12,  0]],

        [[13,  0]],

        [[14,  0]],

        [[15,  0]],

        [[16,  0]],

        [[17,  0]],

        [[18,  0]],

        [[19,  0]]],


       [[[ 0,  1]],

        [[ 1,  1]],

        [[ 2,  1]],

        [[ 3,  1]],

        [[ 4,  1]],

        [[ 5,  1]],

        [[ 6,  1]],

        [[ 7,  1]],

        [[ 8,  1]],

        [[ 9,  1]],

        [[10,  1]],

        [[11,  1]],

        [[12,  1]],

        [[13,  1]],

        [[14,  1]],

        [[15,  1]],

        [[16,  1]],

        [[17,  1]],

        [[18,  1]],

        [[19,  1]]],


       [[[ 0,  2]],

        [[ 1,  2]],

        [[ 2,  2]],

        [[ 3,  2]],

        [[ 4,  2]]

In [28]:
xy_grid = tf.tile(tf.expand_dims(xy_grid, axis=0), [32, 1, 1, 3, 1])
xy_grid

<tf.Tensor: shape=(32, 20, 20, 3, 2), dtype=int32, numpy=
array([[[[[ 0,  0],
          [ 0,  0],
          [ 0,  0]],

         [[ 1,  0],
          [ 1,  0],
          [ 1,  0]],

         [[ 2,  0],
          [ 2,  0],
          [ 2,  0]],

         ...,

         [[17,  0],
          [17,  0],
          [17,  0]],

         [[18,  0],
          [18,  0],
          [18,  0]],

         [[19,  0],
          [19,  0],
          [19,  0]]],


        [[[ 0,  1],
          [ 0,  1],
          [ 0,  1]],

         [[ 1,  1],
          [ 1,  1],
          [ 1,  1]],

         [[ 2,  1],
          [ 2,  1],
          [ 2,  1]],

         ...,

         [[17,  1],
          [17,  1],
          [17,  1]],

         [[18,  1],
          [18,  1],
          [18,  1]],

         [[19,  1],
          [19,  1],
          [19,  1]]],


        [[[ 0,  2],
          [ 0,  2],
          [ 0,  2]],

         [[ 1,  2],
          [ 1,  2],
          [ 1,  2]],

         [[ 2,  2],
          [ 2,  2],


In [29]:
xy_grid = tf.cast(xy_grid, tf.float32)
xy_grid

<tf.Tensor: shape=(32, 20, 20, 3, 2), dtype=float32, numpy=
array([[[[[ 0.,  0.],
          [ 0.,  0.],
          [ 0.,  0.]],

         [[ 1.,  0.],
          [ 1.,  0.],
          [ 1.,  0.]],

         [[ 2.,  0.],
          [ 2.,  0.],
          [ 2.,  0.]],

         ...,

         [[17.,  0.],
          [17.,  0.],
          [17.,  0.]],

         [[18.,  0.],
          [18.,  0.],
          [18.,  0.]],

         [[19.,  0.],
          [19.,  0.],
          [19.,  0.]]],


        [[[ 0.,  1.],
          [ 0.,  1.],
          [ 0.,  1.]],

         [[ 1.,  1.],
          [ 1.,  1.],
          [ 1.,  1.]],

         [[ 2.,  1.],
          [ 2.,  1.],
          [ 2.,  1.]],

         ...,

         [[17.,  1.],
          [17.,  1.],
          [17.,  1.]],

         [[18.,  1.],
          [18.,  1.],
          [18.,  1.]],

         [[19.,  1.],
          [19.,  1.],
          [19.,  1.]]],


        [[[ 0.,  2.],
          [ 0.,  2.],
          [ 0.,  2.]],

         [[ 1.,  2.],


In [35]:
xy_grid = tf.concat([x[:, :, tf.newaxis], y[:, :, tf.newaxis]], axis=-1)
xy_grid

<tf.Tensor: shape=(20, 20, 2), dtype=int32, numpy=
array([[[ 0,  0],
        [ 1,  0],
        [ 2,  0],
        [ 3,  0],
        [ 4,  0],
        [ 5,  0],
        [ 6,  0],
        [ 7,  0],
        [ 8,  0],
        [ 9,  0],
        [10,  0],
        [11,  0],
        [12,  0],
        [13,  0],
        [14,  0],
        [15,  0],
        [16,  0],
        [17,  0],
        [18,  0],
        [19,  0]],

       [[ 0,  1],
        [ 1,  1],
        [ 2,  1],
        [ 3,  1],
        [ 4,  1],
        [ 5,  1],
        [ 6,  1],
        [ 7,  1],
        [ 8,  1],
        [ 9,  1],
        [10,  1],
        [11,  1],
        [12,  1],
        [13,  1],
        [14,  1],
        [15,  1],
        [16,  1],
        [17,  1],
        [18,  1],
        [19,  1]],

       [[ 0,  2],
        [ 1,  2],
        [ 2,  2],
        [ 3,  2],
        [ 4,  2],
        [ 5,  2],
        [ 6,  2],
        [ 7,  2],
        [ 8,  2],
        [ 9,  2],
        [10,  2],
        [11,  2],
        [